In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

In [2]:
ds = xr.open_dataset(f'../GCrundirs/IRF_runs/2016fh_16j_emln_ptegu_0pt1degree_3D_month_01.ncf', engine = "netcdf4")

In [8]:
for m in ['01','04','07','10']:
    ds = xr.open_dataset(f'../GCrundirs/IRF_runs/2016fh_16j_emln_ptegu_0pt1degree_3D_month_{m}.ncf', engine = "netcdf4")
    ds['PEC'] += 1e-11
    ds.to_netcdf(f'../GCrundirs/IRF_runs/2016fh_16j_emln_ptegu_0pt1degree_3D_month_{m}_mod.ncf')
    print(m, 'done')